se cargan las librerias necesarias y se inicia el stemmer nltk

In [1]:
import pandas as pd
import re
import gensim
import numpy as np
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.stem import *
import numpy as np
np.random.seed(2018)
import nltk
import nltk.stem as stemmer
stemmer = PorterStemmer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rober\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

se cargan los documentos generados del webscraping

In [2]:
File = pd.ExcelFile('DatosWoS2020.xlsx')
df=File.parse('Sheet1')

In [3]:
docktype = pd.read_csv('Docktypes.csv')
docktype = docktype["tipo"]

se genera un dataframe de titulos

In [4]:
titles = df["Titulo"]

In [5]:
titles_list = titles.values.tolist()


se limpian los titulos de datos basura y se genera una lista de indices y el dataframe fusion de ambos, es importante cambiar el termino movie por el que se esta trabajando

In [6]:
for i in range(len(titles_list)):
    titles_list[i] = str(titles_list[i])
    titles_list[i] = re.sub("[!,*)@#%(&$_?.^:-]'", '', titles_list[i])
    titles_list[i] = titles_list[i].replace('"','')
    titles_list[i] = titles_list[i].replace("'"," ")
    titles_list[i] = titles_list[i].lower()
    titles_list[i] = titles_list[i].replace("movie","")


In [7]:
indexs = []
for i in range(len(titles_list)):
    indexs.append(i)

In [8]:
df2 = pd.DataFrame(list(zip(titles_list, indexs)),
               columns =['Titulo', 'index'])

se tockeniza el cada titulo sin la palabra topic

In [9]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [10]:
documents = df2
doc_sample = documents[documents['index'] == 0].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['', 'recommendation', 'system', 'based', 'on', '', 'swarm']


 tokenized and lemmatized document: 
['recommend', 'base', 'swarm']


In [11]:
processed_docs = documents['Titulo'].map(preprocess)
processed_docs[:10]

0                           [recommend, base, swarm]
1    [dynam, effect, rat, revenu, viewer, satisfact]
2                                         [descript]
3                     [success, predict, data, mine]
4                                                 []
5                [exploit, charact, network, summar]
6     [tradit, cultur, chines, case, shoot, huizhou]
7                    [feng, xiaogang, economi, oper]
8                [summar, charact, network, analysi]
9    [attribut, trailer, constitut, purchas, intent]
Name: Titulo, dtype: object

se genera un diccionario con las palabras tockenizadas que no pertenezcan a conectores 

In [12]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 base
1 recommend
2 swarm
3 dynam
4 effect
5 rat
6 revenu
7 satisfact
8 viewer
9 descript
10 data


se preparan variables auxiliares y variables que incluyen el topic para la construccion del excel

In [13]:
a = len(processed_docs)
b = len(dictionary)+1
print(a)
print(b)

250
632


In [14]:
newtitles_list = titles.values.tolist()
for i in range(len(newtitles_list)):
    newtitles_list[i] = str(newtitles_list[i])
    newtitles_list[i] = re.sub("[!,*)@#%(&$_?.^:-]'", '', newtitles_list[i])
    newtitles_list[i] = newtitles_list[i].replace('"','')
    newtitles_list[i] = newtitles_list[i].replace("'","")
    newtitles_list[i] = newtitles_list[i].lower()

In [15]:
newindexs = []
for i in range(len(newtitles_list)):
    newindexs.append(i)

In [16]:
newdf2 = pd.DataFrame(list(zip(newtitles_list, newindexs)),
               columns =['Titulo', 'index'])

In [17]:
newdocuments = newdf2
newdoc_sample = newdocuments[newdocuments['index'] == 0].values[0][0]
print('original document: ')
newwords = []
for newword in newdoc_sample.split(' '):
    newwords.append(newword)
print(newwords)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(newdoc_sample))

original document: 
['movie', 'recommendation', 'system', 'based', 'on', 'movie', 'swarm']


 tokenized and lemmatized document: 
['movi', 'recommend', 'base', 'movi', 'swarm']


In [18]:
newlist_titles_processed = []
newprocessed_docs = documents['Titulo'].map(preprocess)
newprocessed_docs[:10]
for i in range(len(newprocessed_docs)):
    newlist_titles_processed.append(' '.join(newprocessed_docs[i]))

se genera una lista vacia y se agregan los titulos

In [19]:
datarray = []
for i in range(a):
    datarray_slice = []
    for j in range(b):
        if  j == 0:
            datarray_slice.append(newtitles_list[i])
        else:
            datarray_slice.append("0")
    datarray.append(datarray_slice)

se agrega un 1 cada vez que se encuentre una coincidencia con las palabras que hay en el texto

In [20]:
for i in range(a):
    for j in range(1,b):
        for k in range(len(newprocessed_docs[i])):
            if dictionary[j-1]  == newprocessed_docs[i][k]:
                datarray[i][j] = "1"

se cuentan cuantas veces aparece cada dayo y se juntan con la palabra en una lista de frecuencias

In [21]:
frcount = []
for i in range(1,len(dictionary)+1):
    count = 0
    for j in range(len(datarray)):
        count = count + int(datarray[j][i])
    namecount = [dictionary[i-1],count]
    frcount.append(namecount)

se ordena la lista en base al numero de repeticiones y se guardan los 15 mayores

In [22]:
sorted_frcount = sorted(frcount, key=lambda x: x[1], reverse=True)
quince_mayores = []
for i in range(15):
    quince_mayores.append(sorted_frcount[i])
print(quince_mayores)

[['base', 37], ['recommend', 31], ['analysi', 23], ['predict', 19], ['learn', 18], ['effect', 12], ['rat', 12], ['trailer', 12], ['genr', 12], ['success', 11], ['model', 11], ['network', 9], ['visual', 9], ['review', 9], ['content', 9]]


se genera una lista con los titulos en los cuales coinciden los datos

In [23]:
coincidence_list = []
procecced_coincidece_list = []
for i in range(len(newtitles_list)):
    for j in range(len(quince_mayores)):
        for k in range(len(newprocessed_docs[i])):
            if quince_mayores[j][0] == newprocessed_docs[i][k]:
                print("la frase " + quince_mayores[j][0] + " aparece en " + titles_list[i])
                if (titles_list[i] in coincidence_list) == False:
                    coincidence_list.append(titles_list[i])
                    procecced_coincidece_list.append(newprocessed_docs[i])
                    break

la frase base aparece en  recommendation system based on  swarm
la frase recommend aparece en  recommendation system based on  swarm
la frase effect aparece en dynamic effects among  ratings,  revenues, and viewer satisfaction
la frase rat aparece en dynamic effects among  ratings,  revenues, and viewer satisfaction
la frase predict aparece en  success prediction using data mining
la frase success aparece en  success prediction using data mining
la frase network aparece en exploiting character networks for  summarization
la frase analysi aparece en  summarization using characters network analysis
la frase network aparece en  summarization using characters network analysis
la frase trailer aparece en attributes in a  trailer that constitutes purchase intention
la frase effect aparece en the effects of  awards on  success: a replication using golden globe data
la frase success aparece en the effects of  awards on  success: a replication using golden globe data
la frase recommend aparece 

se arma la lista de los titulos y 15 ceros en una lista

In [24]:
datacoincidence = []
for i in range(len(coincidence_list)):
    datacoincidence_slice = []
    for j in range(len(quince_mayores)+1):
        if  j == 0:
            datacoincidence_slice.append(newtitles_list[i])
        else:
            datacoincidence_slice.append("0")
    datacoincidence.append(datacoincidence_slice)

se ordena la lista y se suma 1 al valor correspondiente a la lista por cada vez que aparece la palabra de las 15 mayores en el titulo

In [25]:
for i in range(len(coincidence_list)):
    for j in range(1,len(quince_mayores)+1):

        for k in range(len(procecced_coincidece_list[i])):
            if quince_mayores[j-1][0]  == procecced_coincidece_list[i][k]:
                datacoincidence[i][j] = str(int(datacoincidence[i][j]) + 1)

In [26]:
print(datacoincidence[0])

['movie recommendation system based on movie swarm', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']


se generan un dataframe de citaciones y referencias

In [27]:
citations = df["Citas"]

In [28]:
references = df["referencias"]

se une a la lista las citas,referencias, y el tipo de texto

In [29]:
for i in range(len(coincidence_list)):
    for j in range(len(titles_list)):
        if coincidence_list[i] == titles_list[j]:
            datacoincidence[i].append(citations[j])
            datacoincidence[i].append(references[j])
            datacoincidence[i].append(docktype[j])

In [30]:
print(datacoincidence[1])

['dynamic effects among movie ratings, movie revenues, and viewer satisfaction', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '192', 57, 'Article']


se escribe el texto en un excel y en un csv para su posterior analisis

In [31]:
import xlsxwriter
encabezados = []
doctypes = docktype.values.tolist()
for i in range(len(quince_mayores)):
    encabezados.append(quince_mayores[i][0])
encabezados.insert(0,"titulos")
encabezados.append("citas")
encabezados.append("referencias")
encabezados.append("clase")
encabezados = [encabezados]

datacoincidence = encabezados + datacoincidence
workbook = xlsxwriter.Workbook('quincepalabras.xlsx')
worksheet = workbook.add_worksheet()
array = datacoincidence
row = 0

for col, data in enumerate(array):
    worksheet.write_row(col, row, data)

workbook.close()

In [32]:
df3 = pd.DataFrame(datacoincidence,columns=encabezados)
df3 = df3.iloc[1: , :]
df3.to_csv('CsvQuinceMejores.csv',index=False)